<a href="https://colab.research.google.com/github/IsraelAbebe/Amharic-NLP/blob/master/Amharic-Word2Vec/Amharic_Embedding_tf_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Word Embeddings For Amharic Language

## Section  [Using Tensorflow Examples.](https://colab.research.google.com/drive/1aTDUSy8wjA1J07oN2r4Oy12Rgd2yhHDa?authuser=1#scrollTo=V5wlseG_FlDW)


[TF Examples](https://www.tensorflow.org/tutorials/text/word_embeddings)

In [0]:
%tensorflow_version 2.x
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import os
import unicodedata

tfds.disable_progress_bar()

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
embedding_layer = layers.Embedding(1000,10)

In [0]:
DIRECTORY_URL = "gdrive/My Drive/dataset/CACO"
FILE_NAMES = ['gdrive/My Drive/dataset/CACO/CACO_TEXT.txt']

In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  


def unicode_to_ascii(s):
    return tf.strings.strip(''.join(c for c in unicodedata.normalize('NFD', s.numpy().decode('utf8')) if unicodedata.category(c) != 'Mn'))

# Text files
files = tf.data.Dataset.list_files(FILE_NAMES, shuffle=True, seed=None)


labeled_data_sets = []

lines_dataset = tf.data.TextLineDataset(files, compression_type=None, buffer_size=None, num_parallel_reads=None)
labeled_dataset = lines_dataset.map(lambda line: labeler(tf.py_function(unicode_to_ascii, [line], tf.string),0))
labeled_data_sets.append(labeled_dataset)

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 128
TAKE_SIZE = 50000


all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

In [0]:
all_labeled_data

<MapDataset shapes: (<unknown>, ()), types: (tf.string, tf.int64)>

In [0]:
for ex,_ in all_labeled_data.take(5):
  print(ex.numpy().decode('utf8'))

አሁን ወደ ዋናው የጉዞ ፕሮግራም እንመለስ ።
ደብዳቤውን ካነበብክ በኋላ " ጓደኛዬ እኮ እንዲህ አለኝ ! " ብለህ በደስታ ለሌሎች ትናገር ይሆናል ።
የትምህርት ቤቶች እገዛና ድጋፍ ከፍተኛ መሆን እንዳለበትም አስተያየት ሰጥተዋል ።
አብደሻል ?
ደንብ ካልተከበረ የአዲስ አበባ ከተማን ዕድገት ማሳለጥም አይቻልም ።


In [0]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for ex,_ in all_labeled_data:
  some_tokens = tokenizer.tokenize(ex.numpy().decode('utf8'))
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

904412

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [0]:
example_text = next(iter(all_labeled_data))[0].numpy().decode('utf8')
print(example_text)

አሁን ወደ ዋናው የጉዞ ፕሮግራም እንመለስ ።


In [0]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[222004, 338681, 204775, 679989, 429849, 798820]


In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy().decode('utf8'))
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [0]:
train_batch, train_labels = next(iter(train_data))
train_labels.numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Do some Task Directly

In [0]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          14470624  
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 14,470,913
Trainable params: 14,470,913
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_data,
    epochs=10,
    validation_data=test_data, validation_steps=20)

Epoch 1/10
   2788/Unknown - 662s 238ms/step - loss: 0.0209 - accuracy: 1.0000